## Text preprocessing

#### Install required libraries

In [1]:
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import re

from nltk import FreqDist
import itertools

from collections.abc import Sequence
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.tokenize.treebank import TreebankWordDetokenizer

from nltk import word_tokenize
from nltk.lm import MLE, Laplace
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline



from __future__ import unicode_literals
from hazm import *

In [2]:
!pip install hazm
!pip install dadmatools
!pip install pandas
!pip install yake
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade # to update hazm


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached https://github.com/sobhe/hazm/archive/master.zip
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


### read file

In [3]:
file_dir = './data/'

with open(file_dir + 'hp_fa.txt', 'r') as f:
    hp_fa = f.read()

### Part A

A data can be split into sentences using the method sent_tokenize():

In [4]:
hp_fa_tokenized = sent_tokenize(hp_fa)

In [5]:
type(hp_fa_tokenized)

list

In [6]:
print(hp_fa_tokenized[0])

﻿آقا و خانم دورسلي   ساکن خانه شماره چهار خيابان پريوت درايو   بودند. خانواده آنها بسيار معمولي و عادي بود و آن ها از اين بابت بسيار راضي و خوشنود بودند. اين خانواده به هيچ وجه با امور مرموز و اسرار آميز سروکار نداشتند زيرا سحر و جادو را امر مهمل و بيهوده اي مي پنداشتند و علاقه اي به اين گونه مسائل نداشت ن آقاي دورسلي مدير شرکت دريل سازي گرونينگز   ، مردي درشت اندام و قوي هيکل بود با گردني بسيار کوتاه که سبيل بلندي نيز داشت . همسر او، خانم دورسلي زني لاغر اندام بود با موهاي بور و گردني کشيده و بلند. بلندي گردنش بسيار برايش مفيد بود زيرا بيش تر وقتش را صرف سرک کشيدن به خانه ي همسايه ها مي کرد. آنها پسري داشتند به نام دادلي    که به عقيده خودشان لنگه نداشت .   آنها خانواده ي مرفهي بودند و هيچ کم و کسري نداشتند اما در اين خانواده رازي وجود داشت که نبايد برملا مي شد.


Removing additional characters and changing Arabic characters to Farsi

In [7]:
thp = []
for line in hp_fa_tokenized:
    tmp = line
    tmp = re.sub("[۱۲۳۴۵۶۷۸۹۰]*", "", tmp, 1)
    tmp = re.sub("^.$|^.\s|\s.$", " ", tmp,1)
    tmp = re.sub("[\ufeff]", "", tmp, 1)
    tmp = re.sub("^\S*$", " ", tmp)
    tmp = re.sub("([A-Z]|[0-9]|[|]|[a-z])*", " ", tmp, 1)
    thp.append(tmp)
    
hp = ""
for line in thp:
    if line != '':
        hp += line + ' '

First, we normalize the text

In [8]:
hazm_normalizer = Normalizer()
normalized_hp = hazm_normalizer.normalize(hp)

In [9]:
hp_tokens = word_tokenize(normalized_hp)

In [10]:
hp_sentences = normalized_hp.split('.')
print(hp_sentences[0])

آقا و خانم دورسلی ساکن خانه شماره چهار خیابان پریوت درایو بودند


Look at some statistics  of this text.

In [11]:
dataframe = []

rows = ['harry potter']
cols = ['Number of sentences', 'Number of words', 'Number of unique words',\
        'Average word length', 'Average word per sentence']

dataframe.append([len(hp_sentences),len(hp_tokens), len(set(hp_tokens)),\
                  np.sum([len(word) for word in hp_tokens])/len(hp_tokens), \
                 np.sum([len(sentence.split(" ")) for sentence in hp_sentences])/len(hp_sentences)])
 
freq_analysis = pd.DataFrame(dataframe,rows,cols)    
freq_analysis

,Number of sentences,Number of words,Number of unique words,Average word length,Average word per sentence
harry potter,5697,94964,9025,3.545712,16.203616


We have these statistics after removing some stopwords.

To have meaningful sentences, I think we must keep stopwords.

In [12]:
stopwords = [hazm_normalizer.normalize(x.strip()) for x in codecs.open(file_dir + 'stopwords.txt','r','utf-8').readlines()]
hp_nonstop = [t for t in tqdm.tqdm(hp_tokens) if t not in stopwords]

dataframe_nonstop = []

rows = ['harry potter']
cols = ['Number of sentences', 'Number of words', 'Number of unique words', 'Average word length', 'Average word per sentence']

dataframe_nonstop.append([len(hp_sentences),len(hp_nonstop), len(set(hp_nonstop)),\
                  np.sum([len(word) for word in hp_nonstop])/len(hp_nonstop), \
                 np.sum([len(sentence.split(" ")) for sentence in hp_sentences])/len(hp_sentences)])
               
freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop,rows,cols)    
freq_analysis_nonstop


100%|████████████████████████████████| 94964/94964 [00:00<00:00, 2238489.60it/s]


,Number of sentences,Number of words,Number of unique words,Average word length,Average word per sentence
harry potter,5697,60451,8963,4.536253,16.203616


In [13]:
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=23):
    
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

### Part B

In [14]:
n = 2

paddedLine = [list(pad_both_ends(hp_tokens, n))]

train, vocab = padded_everygram_pipeline(n, paddedLine)

lm2 = Laplace(n)

lm2.fit(train, vocab)

print(lm2.counts)

<NgramCounter with 2 ngram orders and 189935 ngrams>


In [19]:
for i in range(0,5):
    print(i+1, ": " , generate_sent(lm2, num_words=random.randint(12,24), random_seed=random.randint(0,100)))

1 :  هیچ وقت هایی که هرجا می‌رود در همه‌ی مهاجمین از هوای ما همه پول نمی‌دم
2 :  رفت به سوراخ شد با شنل سبزی به سوراخ شد با شنل سبزی به سوراخ شد با شنل سبزی به سوراخ شد
3 :  . آن‌قدر ادامه بدین! » . آن‌قدر ادامه بدین! » . آن‌قدر ادامه بدین
4 :  لبخند شیطانی رو شکر کن بسته‌اند در مجاور دیوار می‌کند بیش از غم عمیقی نهفته_بود گفت: -
5 :  و وحشی و وحشی و وحشی و وحشی و وحشی و وحشی و وحشی و


### Part C
The following may happen if the Laplace smoothing method is not used:

>Predictions that are very wrong because of incorrect information or RARE values that exist in the data.

>Predictions may be inaccurate if new occurrences are unknown in the data.

>Smoothing can improve model accuracy by removing the correlation between features.


### Part D and E

In [16]:
n = 3

paddedLine = [list(pad_both_ends(hp_tokens, n))]

train, vocab = padded_everygram_pipeline(n, paddedLine)

lm3 = Laplace(n)

lm3.fit(train, vocab)

print(lm3.counts)

<NgramCounter with 3 ngram orders and 284913 ngrams>


In [20]:
for i in range(0,5):
    print(i+1, ": " , generate_sent(lm3, num_words=random.randint(12,24), random_seed=random.randint(0,100)))

1 :  انداخت . بیش از آن به بعد توی اتاق دومی دادلی باشی . اونا
2 :  شنل می‌توانست به سرعت جلو رفتند و سالن عمومی شدند . هاگرید لبخندی بر لب هاگرید نشست و منتظر می‌مونم تا فیلچ بیاد و
3 :  گفت و همراه نویل به چهره‌های خطاکار آن‌ها نگاه کرد که کنار پاشه چیه ؟ پیش از مسابقه چشم بردارد به نویل گفت :
4 :  . آن خانه یک اتاق‌خواب بیش‌تر نداشت . آقای اولیوندر ادامه داد: - اسلیترین صاحب توپه.
5 :  سوی رون رفت که به طرف صدای بدعنق می‌رفت می‌شنیدند . اسنیپ روبه روی شبحی که یقه‌ی لباس


In [21]:
n = 5

paddedLine = [list(pad_both_ends(hp_tokens, n))]

train, vocab = padded_everygram_pipeline(n, paddedLine)

lm5 = Laplace(n)

lm5.fit(train, vocab)

print(lm5.counts)

<NgramCounter with 5 ngram orders and 474890 ngrams>


In [ ]:
for i in range(0,5):
    print(i+1, ": " , generate_sent(lm5, num_words=random.randint(12,24), random_seed=random.randint(0,100)))

1 :  به خودش اشاره کرد . رون گفت: - تو به ده تا مالفوی می‌ارزی . مگه کلاه گروه‌بندی
2 :  تکیه داد و مشتاقانه به هری خیره شد . پس از نیم ساعت
3 :  مثل مار درازی در هوا پیچ و تاب می‌خورد به سرعت از آن جا رفت . هرمیون که در فکر بود گفت: -
4 :  شنیدن حر‌های هاگرید دوربین او را گرفت و شروع به پاک کردن بینیاش کرد . پسر
